In [39]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import filedialog
import time



# ask the user to select a file

# Ask user to select a file
root = tk.Tk()
root.withdraw()
print("Select a CSV file to load into a Pandas DataFrame")
file_path = filedialog.askopenfilename(filetypes=[('CSV files', '*.csv')])

# Load selected file into Pandas DataFrame
df = pd.read_csv(file_path)

# only keep columns we need
df = df[['Supplier Name', 'Supplier Number', 'Payment Date', 'Payment Total', 'Po Number', 'Invoice Number', 'Invoice Date', ]]

# Convert the columns to formats we need
df['Payment Date'] = pd.to_datetime(df['Payment Date'])
df['Payment Total'] = pd.to_numeric(df['Payment Total'].str.replace(',', ''), errors='coerce')
df['Supplier Number'] = pd.to_numeric(df['Supplier Number'], errors='coerce')
df['Po Number'] = pd.to_numeric(df['Po Number'], errors='coerce')


# print total number of rows and columns
print("Total number of rows:", len(df))
print("Total number of columns:", len(df.columns))
#print a table of columns and data types
print(df.dtypes)

# Filter data for transactions over 1000 and sort by supplier name  
filtered_df = df[df['Payment Total'] > 1000].sort_values(by=['Supplier Name'])
#print number of rows 
print("Number of rows after filtering:", len(filtered_df))

#sort by supplier name
filtered_df = filtered_df.sort_values(by=['Supplier Name'])

#print the first 50 rows
print(filtered_df.head(50))

#choose the first 10000 rows
#filtered_df = filtered_df.head(10000)



Select a CSV file to load into a Pandas DataFrame


C:\Users\Alex\AppData\Local\Temp\ipykernel_2644\2209136382.py:19: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Total number of rows: 510999
Total number of columns: 7
Supplier Name              object
Supplier Number             int64
Payment Date       datetime64[ns]
Payment Total             float64
Po Number                 float64
Invoice Number             object
Invoice Date               object
dtype: object
Number of rows after filtering: 124950
                             Supplier Name  Supplier Number Payment Date  \
434103                   1-800-Bollard Inc           875254   2020-12-08   
418372                   1-800-Bollard Inc           875254   2020-12-08   
69637                    1-800-Bollard Inc           875254   2021-02-16   
199049                   1-800-Bollard Inc           875254   2020-12-08   
388817                1211 Hixson Pike LLC           874115   2019-02-13   
5992                         1400 Chestnut           875548   2021-06-29   
397691                 201 Main Street LLC           871089   2016-05-23   
87970                  201 Main Street LLC   

Load file, convert payment date and total to data types
Check total rows and columns and datatypes

In [40]:
def within_1_percent(df):
    """
    Filter data within 1% of each other in the Payment Total column and within 5 days of each other.
    """
    if len(df) >= 2:
        mean_payment = df['Payment Total'].mean()
        upper_limit = mean_payment * 1.01
        lower_limit = mean_payment * 0.99
        max_date = df['Payment Date'].max()
        min_date = df['Payment Date'].min()
        if (max_date - min_date).days <= 5:
            return df[(df['Payment Total'] <= upper_limit) & (df['Payment Total'] >= lower_limit)]
        else:
            return pd.DataFrame()
    else:
        return pd.DataFrame()
        
# Filter data for transactions over 1000
filtered_df = df[df['Payment Total'] > 1000]

#choose the first 10000 rows
#filtered_df = filtered_df.head(10000)

# Group by Supplier Name and filter for transactions within 1% of each other in the Payment Total column
grouped_df = filtered_df.groupby('Supplier Name').apply(within_1_percent).reset_index(drop=True)

# Exclude results with less than two rows per vendor
grouped_df = grouped_df.groupby('Supplier Name').filter(lambda x: len(x) >= 2)

# print total number of rows 
print("Total number of rows:", len(grouped_df))

# Remove empty dataframes from the grouped data
grouped_df = grouped_df[grouped_df['Payment Total'].notna()]

#print first 50 rows of the grouped data just supplier, amount, date and invoice number
print("First 50 rows of the grouped data same supplier, within 1% and 5 days")
print(grouped_df[['Supplier Number', 'Payment Total', 'Payment Date', 'Invoice Number']].head(50))

Total number of rows: 375
First 50 rows of the grouped data same supplier, within 1% and 5 days
    Supplier Number  Payment Total Payment Date Invoice Number
1          732743.0        5895.43   2012-04-18     7101332853
2          732743.0        5895.43   2012-04-18     7101332853
3          867619.0        1301.85   2014-05-20          79095
4          867619.0        1301.85   2014-05-20          79095
5          871874.0        1750.80   2017-04-10         035113
6          871874.0        1750.80   2017-04-10         035113
7          866643.0        2010.15   2013-05-31          25578
8          866643.0        2010.15   2013-05-31          25578
9          874759.0        3490.00   2020-02-19         425081
10         874759.0        3490.00   2020-02-19         425079
11         869725.0       22379.99   2015-12-07       15-17689
12         869725.0       22379.99   2015-12-07       15-17689
13         874539.0        1097.00   2019-09-09       30008143
14         874539.0   

Filter for 1000
Filter for Same Supplier, +-5d Payment Date, and 1% payment total 

In [41]:
# Define a function to filter groups with less than 2 rows
def at_least_2_rows(df):
    return len(df) >= 2

# Group the data by Invoice Number and Payment Total, and filter for groups with at least 2 rows and sort by supplier name
grouped_df2 = df.groupby(['Invoice Number', 'Payment Total']).filter(at_least_2_rows).sort_values(by=['Supplier Name'])

# Remove empty dataframes from the grouped data
grouped_df2 = grouped_df2[grouped_df2['Payment Total'].notna()]

# Print total number of rows
print("Total number of rows:", len(grouped_df2))

#print first 50 rows of the grouped data just supplier, amount, date and invoice number
print("First 50 rows of the grouped data same invoice, same amount")
print(grouped_df2[['Supplier Number', 'Payment Total', 'Payment Date', 'Invoice Number']].head(50))



Total number of rows: 36645
First 50 rows of the grouped data same invoice, same amount
        Supplier Number  Payment Total Payment Date  Invoice Number
418372           875254        7379.23   2020-12-08            4082
199049           875254        7379.23   2020-12-08            4082
434103           875254        7379.23   2020-12-08            4082
298289           731922        1000.00   2015-02-19     COC01302015
231181           874550         150.00   2019-09-16             004
76233             96999        1845.00   2021-07-22      9411836016
476603            96999         102.99   2013-01-22         TP78542
297903            96999         102.99   2013-01-22         TP78542
71052             96999        1845.00   2021-07-22      9411836016
328699            96999         144.19   2012-08-13         TP58323
415593            96999         144.19   2012-08-13         TP58323
255370           743050        5129.43   2015-10-26         4132115
346292           743050     

Matches for invoice number and total

In [42]:

# Group the data by PO Number and Payment Total, and filter for groups with at least 2 rows and sort by supplier name
grouped_df3 = filtered_df.groupby(['Po Number', 'Payment Total']).filter(at_least_2_rows).sort_values('Po Number')

# Remove empty dataframes from the grouped data
grouped_df3 = grouped_df3[grouped_df3['Payment Total'].notna()]

# Print total number of rows
print("Total number of rows:", len(grouped_df3))

# print first 50 rows of the grouped data just supplier, amount, date and invoice number
print("First 50 rows of the grouped data same PO, same amount")
print(grouped_df3[['Supplier Number', 'Payment Total', 'Payment Date', 'Invoice Number']].head(50))


Total number of rows: 20053
First 50 rows of the grouped data same PO, same amount
        Supplier Number  Payment Total Payment Date Invoice Number
315382           555101        3735.67   2012-01-19          15289
268810           555101        3735.67   2012-01-19          15411
197511           555101        3735.67   2012-03-16          15539
366655           747933        1088.00   2012-05-15     9423588507
56355            747933        1088.00   2012-05-15     9423604659
179607           151363        3688.00   2012-04-30      BMS190235
487512           151363        5532.00   2012-03-12      BMS154769
316657           151363        3688.00   2012-03-23      BMS179958
488019           151363        4610.00   2012-05-14      BMS207911
177600           151363        3688.00   2012-03-23      BMS171118
177069           151363        5532.00   2012-02-01      BMS147248
176477           151363        4149.00   2012-07-12      BMS247358
321440           151363        5532.00   2012-

Matches for Po Number and total

In [43]:
# Save the Excel file
print("Saving Excel file...")


# Create a Tkinter file save dialog box with default extension '.xlsx'
file_path = filedialog.asksaveasfilename(defaultextension='.xlsx', filetypes=[('Excel files', '*.xlsx')])

# Create an Excel writer object
writer = pd.ExcelWriter(file_path, engine='xlsxwriter')

# Write the filtered data to a tab
filtered_df.to_excel(writer, sheet_name='Filtered Data', index=False)

# Write the grouped data to a tab
grouped_df.to_excel(writer, sheet_name='Grouped Data', index=False)

# Write the grouped data to a tab
grouped_df2.to_excel(writer, sheet_name='Grouped Data 2', index=False)

# Write the grouped data to a tab
grouped_df3.to_excel(writer, sheet_name='Grouped Data 3', index=False)

print("Excel file saved successfully!")

# Close the Excel writer object
writer.close()

# open the file that was just saved
import os
os.startfile(file_path)



Saving Excel file...
Excel file saved successfully!
